In [4]:
! pip install --quiet fuzzywuzzy[speedup]

In [95]:
import pandas as pd
import numpy as np
import json

In [3]:
with open('./data/sample.jl', 'r') as f:
    jobs = pd.DataFrame([json.loads(d) for d in f])

In [4]:
direct_match = (pd
                .read_excel('./data/SOC/soc_2010_direct_match_title_file.xls', skiprows=6)
                .rename(columns = {'2010 SOC Direct Match Title': 'title', 
                                     '2010 SOC Code': 'code'})[['title', 'code']])

samples = (pd
           .read_excel('./data/SOC/Sample of Reported Titles.xlsx')
           .rename(columns = {'Reported Job Title': 'title', 
                              'O*NET-SOC Code': 'code'})[['title', 'code']])

alternates = (pd
              .read_excel('./data/SOC/Alternate Titles.xlsx')
              .rename(columns = {'Alternate Title': 'title', 
                                 'O*NET-SOC Code': 'code'})[['title', 'code']])

alternates['code'] = alternates.code.str.slice(0,7)
samples['code'] = samples.code.str.slice(0,7)

In [126]:
lookup = pd.concat([direct_match, samples, alternates]).drop_duplicates(['code', 'title'])
lookup['title'] = lookup.title.str.lower()

# filter all titles that have more than one match with different codes!
idx = lookup.groupby('title').transform('count').code
lookup = lookup[idx == 1] 

In [127]:
lookup.to_csv('crosswalks/soc-title-lookup.csv', index=False)

In [128]:
jobs['title'] = jobs.title.str.lower()
j = jobs.merge(lookup, how='left', on = ['title'])

In [129]:
# Percent found in exact string match
j[~j.code.isna()].shape[0] / j.shape[0]

0.107

In [ ]:
j[~j.code.isna()]

### Explore matching with partial matches

In [50]:
def lookup_two(lookup):
    def f(t):
        z = zip(lookup.code, lookup.title)
        split = t.split(' ')
        options = [(code, title) for code,title in z if title in t]
        if len(options) > 0:
            options = sorted(options, key = lambda t: -len(t[1]))
        return options[0] if options else (None,None)
    return f

In [81]:
from fuzzywuzzy import fuzz, utils, process

def lookup_code(lookup):
    def f(t):
        try:
            prop, score = process.extractOne(t, lookup.title, score_cutoff = 0)
            return prop
        except ValueError:
            return None
    return f

In [131]:
leftovers = j[j.code.isnull()].reset_index(drop=True)
codes, titles = zip(*leftovers.title.map(lookup_two(lookup)))
leftovers = leftovers.assign(code=codes, assigned_title = titles)

In [133]:
leftovers[['title', 'assigned_title', 'code']]

,title,assigned_title,code
0,health info clerk-medical records,None,None
1,real estate processor,None,None
2,unit coordinator / cna-i,unit coordinator,39-9041
3,aircraft maintenance training instructor,None,None
4,maintenance shift manager,None,None
5,maintenance training instructor,None,None
6,director of quality-maintenance,director of quality,11-3051
7,"general manager, vip seating",None,None
8,line service technician,None,None
9,"general manager, vip seating",None,None


In [ ]:
t = utils.full_process(j[j.code.isnull()].title.values[5])
lookup_two(lookup)(t)